In [11]:
import numpy as np
import pandas as pd
from typing import Optional, Tuple
import sys

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from pickle import dump
from sklearn.preprocessing import MinMaxScaler
import time
from tqdm import tqdm

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("cuda is available")
else:
    print("cuda is NOT available")


import shutil
import warnings
import pickle

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

from nn_functions import surrogate
from moving_average import moving_average_1d
import copy
from GAMMA_obj_temp_depth import GAMMA_obj

import sys
sys.path.append('../1_model')
from TiDE import TideModule, quantile_loss, TiDE_forward


cuda is available


In [12]:
INPUT_DATA_DIR = "data"
SIM_DIR_NAME = "single_track_square"
BASE_LASER_FILE_DIR = "laser_power_profiles/csv"
CLOUD_TARGET_BASE_PATH = "result"
solidus_temp = 1600
window = 50
sim_interval = 5
init_runs = 60 #50 

GAMMA_class = GAMMA_obj(INPUT_DATA_DIR, SIM_DIR_NAME, BASE_LASER_FILE_DIR, CLOUD_TARGET_BASE_PATH, solidus_temp, window, init_runs, sim_interval,laser_power_number=1)
init_avg = GAMMA_class.run_initial_steps()
init_avg = torch.tensor(init_avg,dtype=torch.float32)[:,-window:] # shape = [2,50]

100%|██████████| 300/300 [00:06<00:00, 45.14it/s]


In [13]:
df_one_print = pd.read_csv('single_track_ref.csv')

loc_X_list = df_one_print["X"].to_numpy().reshape(-1,1)
loc_Y_list = df_one_print["Y"].to_numpy().reshape(-1,1)
loc_Z_list = df_one_print["Z"].to_numpy().reshape(-1,1)
dist_X_list = df_one_print["Dist_to_nearest_X"].to_numpy().reshape(-1,1)
dist_Y_list = df_one_print["Dist_to_nearest_Y"].to_numpy().reshape(-1,1)
scan_spd_list = df_one_print["scanning_speed"].to_numpy().reshape(-1,1)

# laser on/off indicator
laser_on_off = df_one_print["laser_power_number"].to_numpy().reshape(-1,1)

# laser power
laser_power_ref = torch.tensor(df_one_print["Laser_power"].to_numpy().reshape(-1,1),dtype=torch.float32)
laser_power_past = laser_power_ref[:window]

# fix_covariates = torch.tensor(np.concatenate((loc_X_list,loc_Y_list,loc_Z_list,dist_X_list,dist_Y_list,scan_spd_list, laser_on_off),axis=1),dtype=torch.float32)
fix_covariates = torch.tensor(np.concatenate((loc_Z_list,dist_X_list,dist_Y_list),axis=1),dtype=torch.float32)

# temporary ref
# apply moving average for mp temp
mp_temp_raw = df_one_print["melt_pool_temperature"].to_numpy()
mp_temp_mv = moving_average_1d(mp_temp_raw,4)
mp_temp = copy.deepcopy(mp_temp_raw)
mp_temp[1:-2] = mp_temp_mv
mp_temp = mp_temp

mp_temp_ref = torch.tensor(mp_temp,dtype=torch.float32)

In [14]:
print("laser_power_ref shape:", laser_power_ref.size())
print("laser_power_ref min:", torch.min(laser_power_ref).item())
print("laser_power_ref max:", torch.max(laser_power_ref).item())

print("fix_covariates shape:", fix_covariates.size())

col_min = torch.min(fix_covariates, dim=0).values
col_max = torch.max(fix_covariates, dim=0).values

print("fix_covariates column-wise min:", col_min.cpu().numpy())
print("fix_covariates column-wise max:", col_max.cpu().numpy())

nan_indices = torch.nonzero(torch.isnan(mp_temp_ref), as_tuple=True)[0]
print("NaN indices:", nan_indices)

valid_values = mp_temp_ref[~torch.isnan(mp_temp_ref)]

print("mp_temp_ref (valid) min:", valid_values.min().item())
print("mp_temp_ref (valid) max:", valid_values.max().item())



laser_power_ref shape: torch.Size([6295, 1])
laser_power_ref min: 531.5235595703125
laser_power_ref max: 600.7783203125
fix_covariates shape: torch.Size([6295, 3])
fix_covariates column-wise min: [0.   0.75 0.75]
fix_covariates column-wise max: [ 7.5 20.  20. ]
NaN indices: tensor([6292, 6294])
mp_temp_ref (valid) min: 436.1014099121094
mp_temp_ref (valid) max: 3834.751220703125


In [15]:
import torch

# values from user
x_min = torch.tensor([[0.0, 0.75, 0.75, 504.26]], dtype=torch.float32).to(device)
x_max = torch.tensor([[7.5, 20.0, 20.0, 732.298]], dtype=torch.float32).to(device)

y_min = torch.tensor([[436.608, -0.559]], dtype=torch.float32).to(device)
y_max = torch.tensor([[4509.855, 0.551]], dtype=torch.float32).to(device)


In [16]:
def normalize_x(x, dim_id):
    x_min_selected = x_min[0, dim_id]
    x_max_selected = x_max[0, dim_id]
    return 2 * (x - x_min_selected) / (x_max_selected - x_min_selected) - 1

def inverse_normalize_x(x_norm, dim_id):
    x_min_selected = x_min[0, dim_id]
    x_max_selected = x_max[0, dim_id]
    return 0.5 * (x_norm + 1) * (x_max_selected - x_min_selected) + x_min_selected

def normalize_y(y, dim_id):
    y_min_selected = y_min[0, dim_id]
    y_max_selected = y_max[0, dim_id]
    return 2 * (y - y_min_selected) / (y_max_selected - y_min_selected) - 1

def inverse_normalize_y(y_norm, dim_id):
    y_min_selected = y_min[0, dim_id]
    y_max_selected = y_max[0, dim_id]
    return 0.5 * (y_norm + 1) * (y_max_selected - y_min_selected) + y_min_selected


In [17]:
def run_one_step_policy(GAMMA_obj, policy_model, P, window):
    # Reference trajectory for temperature (original scale)
    mp_temp_ref = GAMMA_obj.ref[GAMMA_obj.MPC_counter:GAMMA_obj.MPC_counter + P]  # [50]

    '''
    print("mp_temp_ref shape:", mp_temp_ref.shape)
    print("mp_temp_ref:", mp_temp_ref[:,50,:])
    '''

    mp_temp_ref_t = torch.as_tensor(mp_temp_ref, dtype=torch.float32, device=device).reshape(1, P, 1)  # [1, P, 1]

    # Past input (original scale)
    mp_temp_past_t = GAMMA_obj.x_past.T.unsqueeze(0).to(device)  # [1, 50, 2]
    laser_past_t = GAMMA_obj.u_past.view(1, -1, 1).to(device)     # [1, 50, 1]

    '''
    print("mp_temp_ref_t shape:", mp_temp_ref_t.shape)  # [1, 50, 1]
    print("mp_temp_ref_t:", mp_temp_ref_t[:,50,:])
    print("laser_past_t shape:", laser_past_t.shape)  # [1, 50, 1]
    print("laser_past_t:", laser_past_t[:,50,:])
    '''

    fix_cov_past = GAMMA_obj.fix_cov_all[GAMMA_obj.MPC_counter - window:GAMMA_obj.MPC_counter, :]
    fix_cov_past_t = torch.as_tensor(fix_cov_past, dtype=torch.float32, device=device).unsqueeze(0)  # [1, 50, 3]

    # Normalize
    fix_cov_past_s = normalize_x(fix_cov_past_t, dim_id=[0, 1, 2])    # assume features 0~2 in x
    laser_past_s = normalize_x(laser_past_t, dim_id=[3])             # laser power at feature 3
    mp_temp_past_s = normalize_y(mp_temp_past_t, dim_id=[0, 1])       # temp and depth
    policy_in_past = torch.cat((fix_cov_past_s, laser_past_s, mp_temp_past_s), dim=2)  # [1, 50, 6]

    # Future covariates
    fix_cov_future = GAMMA_obj.fix_cov_all[GAMMA_obj.MPC_counter:GAMMA_obj.MPC_counter + P, :]
    fix_cov_future_t = torch.as_tensor(fix_cov_future, dtype=torch.float32, device=device).unsqueeze(0)  # [1, P, 3]
    fix_cov_future_s = normalize_x(fix_cov_future_t, dim_id=[0, 1, 2])
    mp_temp_ref_s = normalize_y(mp_temp_ref_t, dim_id=[0])[:, :, 0].unsqueeze(-1)

    # Constraints
    depth_upper_const = 0.4126
    depth_lower_const = 0.1423
    y_const_t = torch.tensor([[depth_upper_const, depth_lower_const]] * P, dtype=torch.float32, device=device).reshape(1, P, 2)
    y_const_s = normalize_y(y_const_t, dim_id=[1])  # assume dim=1 is depth

    policy_in_future = torch.cat((fix_cov_future_s, mp_temp_ref_s, y_const_s), dim=2)  # [1, P, 6]

    # Policy inference
    print("policy_in_past shape:", policy_in_past.shape)  # [1, 50, 6]
    print("policy_in_future shape:", policy_in_future.shape)  # [1, P, 6]
    print("policy_in_past_first component:", policy_in_past[0, 0, :])  # Last past input
    print("policy_in_future_first component:", policy_in_future[0, 0, :])  # First future input
    print("policy_in_past_last component:", policy_in_past[0, -1, :])  # Last past input
    print("policy_in_future_last component:", policy_in_future[0, -1, :])  # First future input

    u_pred = policy_model((policy_in_past, policy_in_future))
    print("u_pred shape:", u_pred.shape)  # [1, P, 1]
    print("u_pred:", u_pred.T)  

    u_first = u_pred[0,0]
    u_applied = float(inverse_normalize_x(u_first, dim_id=[3]))  # laser power

    # Simulate one step
    x_current, depth_current = GAMMA_obj.run_sim_interval(u_applied)

    # Update past sequence
    GAMMA_obj.x_past[:, :-1] = GAMMA_obj.x_past[:, 1:]
    GAMMA_obj.x_past[0, -1] = x_current
    GAMMA_obj.x_past[1, -1] = depth_current

    GAMMA_obj.u_past[:-1] = GAMMA_obj.u_past[1:].clone()
    GAMMA_obj.u_past[-1] = u_applied

    # Save state
    GAMMA_obj.MPC_counter += 1

    # FIXED: device-matched saving
    new_state = torch.tensor([[x_current, depth_current]], device=GAMMA_obj.x_past_save.device)
    GAMMA_obj.x_past_save = torch.cat((GAMMA_obj.x_past_save, new_state), dim=0)

    new_u = torch.tensor([[u_applied]], device=GAMMA_obj.u_past_save.device)
    GAMMA_obj.u_past_save = torch.cat((GAMMA_obj.u_past_save, new_u), dim=0)

In [18]:
import matplotlib.pyplot as plt
import numpy as np

def plot_fig(MPC_GAMMA, N_step, save_path=None):
    plt.figure(figsize=[8, 6])

    plt.subplot(3, 1, 1)
    plt.plot(MPC_GAMMA.x_past_save[:N_step, 0], label="GAMMA simulation")
    plt.plot(MPC_GAMMA.ref[:N_step], label="Reference")
    plt.legend()
    plt.xlabel("MPC time step (*** sec/iteration)")
    plt.ylabel("Melt Pool Temperature (K)")

    plt.subplot(3, 1, 2)
    plt.plot(MPC_GAMMA.x_past_save[:N_step, 1], label="GAMMA simulation")
    plt.plot(np.linspace(0, N_step, N_step), 0.225 * np.ones(N_step), linestyle='--', label="Upper Bound")
    plt.plot(np.linspace(0, N_step, N_step), 0.075 * np.ones(N_step), linestyle='--', label="Lower Bound")
    plt.xlabel("MPC time step (*** sec/iteration)")
    plt.ylabel("Melt Pool Depth (mm)")
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(MPC_GAMMA.u_past_save[:N_step])
    plt.ylabel("Laser Power (W)")
    plt.xlabel("MPC time step (*** sec/iteration)")
    plt.xlim(0, N_step)

    if save_path:
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
        print(f"Saved plot to {save_path}")
    else:
        plt.tight_layout()
        plt.show()


In [19]:
print(laser_power_past.size())
print(laser_power_past)



torch.Size([50, 1])
tensor([[533.8476],
        [533.8514],
        [533.8552],
        [533.8588],
        [533.8624],
        [533.8660],
        [533.8694],
        [533.8729],
        [533.8762],
        [533.8795],
        [533.8827],
        [533.8858],
        [533.8889],
        [533.8918],
        [533.8948],
        [533.8976],
        [533.9004],
        [533.9031],
        [533.9058],
        [533.9083],
        [533.9108],
        [533.9133],
        [533.9156],
        [533.9180],
        [533.9202],
        [533.9224],
        [533.9245],
        [533.9265],
        [533.9285],
        [533.9304],
        [533.9323],
        [533.9340],
        [533.9357],
        [533.9374],
        [533.9390],
        [533.9404],
        [533.9419],
        [533.9433],
        [533.9446],
        [533.9459],
        [533.9470],
        [533.9482],
        [533.9492],
        [533.9502],
        [533.9511],
        [533.9520],
        [533.9528],
        [533.9536],
        [533.9542],


In [20]:
import importlib
import policy
importlib.reload(policy)
from policy import PolicyNN

from tqdm import tqdm
import torch

P = 50
window = 50
N_step = len(mp_temp_ref) - init_runs + P

model = PolicyNN(
    past_input_dim=6,
    future_input_dim=6,
    output_dim=1,
    p=P,
    window=window,
    hidden_dim=1024,
    n_layers=3,
    dropout_p=0.1
).to(device)

model_path = "/home/ftk3187/github/DPC_research/02_DED/2_policy/trainresults/policy_model_0722_3L_1024H_s0_c0.pth"
# model_path = "/home/ftk3187/github/DPC_research/02_DED/2_policy/trainresults/policy_model_0716_epoch100_3L_1024H_s2_c2.pth"
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

# 초기화
GAMMA_class.ref = mp_temp_ref[10:].clone()
GAMMA_class.fix_cov_all = fix_covariates[10:].clone()
GAMMA_class.x_past = init_avg[:,-50:].clone()
GAMMA_class.u_past = laser_power_past[-50:].clone()

GAMMA_class.x_hat_current = GAMMA_class.x_past[:, -1]
GAMMA_class.x_sys_current = GAMMA_class.x_past[:, -1].reshape(2, 1)

GAMMA_class.x_past_save = GAMMA_class.x_past.T.clone()
GAMMA_class.u_past_save = GAMMA_class.u_past.clone()
GAMMA_class.MPC_counter = window

# 실행 루프
for i in tqdm(range(N_step - P)):
    run_one_step_policy(GAMMA_class, model, P=P, window=window)

    # 원하는 구간마다 저장
    if i % 1000 == 0:
        print(GAMMA_class.u_past_save.size())
        print(GAMMA_class.u_past_save)
        # plot_fig(GAMMA_class, N_step)


  0%|          | 0/6235 [00:00<?, ?it/s]

policy_in_past shape: torch.Size([1, 50, 6])
policy_in_future shape: torch.Size([1, 50, 6])
policy_in_past_first component: tensor([-0.8000, -0.5842, -1.0000, -0.7405, -0.0450, -0.7402], device='cuda:0')
policy_in_future_first component: tensor([-0.8000,  0.7154, -1.0000, -0.0175,  0.7506,  0.2636], device='cuda:0')
policy_in_past_last component: tensor([-8.0000e-01,  6.8942e-01, -1.0000e+00, -7.3956e-01,  4.0293e-04,
        -6.7398e-01], device='cuda:0')
policy_in_future_last component: tensor([-0.8000,  0.0109, -1.0000, -0.0167,  0.7506,  0.2636], device='cuda:0')
u_pred shape: torch.Size([1, 50, 1])
u_pred: tensor([[[ 0.2977],
         [ 0.7053],
         [ 0.9353],
         [ 1.6759],
         [ 5.2705],
         [ 2.5240],
         [ 2.9497],
         [ 4.7353],
         [ 4.6564],
         [ 4.5267],
         [-1.7325],
         [ 0.2982],
         [-0.3515],
         [-1.2579],
         [-0.9179],
         [-0.7537],
         [ 0.0094],
         [-0.0702],
         [ 6.0531],
 

  0%|          | 1/6235 [00:00<12:37,  8.23it/s]

torch.Size([51, 1])
tensor([[533.8476],
        [533.8514],
        [533.8552],
        [533.8588],
        [533.8624],
        [533.8660],
        [533.8694],
        [533.8729],
        [533.8762],
        [533.8795],
        [533.8827],
        [533.8858],
        [533.8889],
        [533.8918],
        [533.8948],
        [533.8976],
        [533.9004],
        [533.9031],
        [533.9058],
        [533.9083],
        [533.9108],
        [533.9133],
        [533.9156],
        [533.9180],
        [533.9202],
        [533.9224],
        [533.9245],
        [533.9265],
        [533.9285],
        [533.9304],
        [533.9323],
        [533.9340],
        [533.9357],
        [533.9374],
        [533.9390],
        [533.9404],
        [533.9419],
        [533.9433],
        [533.9446],
        [533.9459],
        [533.9470],
        [533.9482],
        [533.9492],
        [533.9502],
        [533.9511],
        [533.9520],
        [533.9528],
        [533.9536],
        [533.9542],


  0%|          | 2/6235 [00:00<12:21,  8.41it/s]

...

  4%|▎         | 224/6235 [00:26<12:01,  8.33it/s]

policy_in_past shape: torch.Size([1, 50, 6])
policy_in_future shape: torch.Size([1, 50, 6])
policy_in_past_first component: tensor([-0.8000, -1.0000,  0.7619,  0.0824,  0.2895, -0.5278], device='cuda:0')
policy_in_future_first component: tensor([-0.8000, -1.0000, -0.5378, -0.0012,  0.7506,  0.2636], device='cuda:0')
policy_in_past_last component: tensor([-0.8000, -1.0000, -0.5118, -0.4354,  0.1536, -0.5664], device='cuda:0')
policy_in_future_last component: tensor([-0.8000, -0.1886, -1.0000,  0.0113,  0.7506,  0.2636], device='cuda:0')
u_pred shape: torch.Size([1, 50, 1])
u_pred: tensor([[[-0.4272],
         [-0.3459],
         [-0.0131],
         [ 0.6036],
         [ 2.3954],
         [ 1.8084],
         [ 2.5919],
         [ 4.3634],
         [ 4.7631],
         [ 4.8798],
         [-0.8994],
         [ 0.7635],
         [-0.2206],
         [-1.1531],
         [-0.9028],
         [-0.6991],
         [ 0.1562],
         [-0.4010],
         [ 6.8741],
         [ 5.5756],
         [ 1.

  4%|▎         | 225/6235 [00:26<11:55,  8.40it/s]


policy_in_past shape: torch.Size([1, 50, 6])
policy_in_future shape: torch.Size([1, 50, 6])
policy_in_past_first component: tensor([-0.8000, -1.0000,  0.7359,  0.0747,  0.2826, -0.5251], device='cuda:0')
policy_in_future_first component: tensor([-0.8000, -1.0000, -0.5638,  0.0041,  0.7506,  0.2636], device='cuda:0')
policy_in_past_last component: tensor([-0.8000, -1.0000, -0.5378, -0.4272,  0.1393, -0.5637], device='cuda:0')
policy_in_future_last component: tensor([-0.8000, -0.1626, -1.0000,  0.0070,  0.7506,  0.2636], device='cuda:0')
u_pred shape: torch.Size([1, 50, 1])
u_pred: tensor([[[-0.3992],
         [-0.3189],
         [ 0.0106],
         [ 0.6346],
         [ 2.4757],
         [ 1.8434],
         [ 2.6126],
         [ 4.3546],
         [ 4.7583],
         [ 4.8798],
         [-0.9069],
         [ 0.7635],
         [-0.2158],
         [-1.1484],
         [-0.9008],
         [-0.6997],
         [ 0.1531],
         [-0.4012],
         [ 6.8511],
         [ 5.5453],
         [ 1.

KeyboardInterrupt: 